In [18]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import sys
sys.path.append(r'DL-objectclassification\LeNet')
from model import LeNet
import pandas as pd
import os

In [19]:
PRED_PATH = 'E:\\Dataset\\mask\\test'

In [20]:
transform = transforms.Compose(
    [transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [21]:
classes = ('mask_weared_incorrect', 'mask_weared_incorrect', 'without_mask')

In [22]:
net = LeNet()
net.fc3 = nn.Linear(84, len(classes))
net.load_state_dict(torch.load('weights\Lenet2.pth'))

<All keys matched successfully>

In [23]:
def getFileList(path):
    for root, dirs, files in os.walk(path):
        return files
file_name = getFileList(PRED_PATH)
del(file_name[0])

In [24]:
y_pre = []
for step, img in enumerate(file_name,start=0):
    im = Image.open(os.path.join(PRED_PATH,img))
    im = transform(im)  # [C, H, W]
    im = torch.unsqueeze(im, dim=0)  # [N, C, H, W]
    with torch.no_grad():
        outputs = net(im)
        predict = torch.max(outputs, dim=1)[1].numpy()
    y_pre.append(classes[int(predict)])

In [25]:
#----------------结果输出----------------
result=pd.read_csv('submit/sample_submit.csv')
result['label']=y_pre
result.to_csv('submit/result-de2.csv',index=False)